In [1]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cv-3-bitcoin (use `wandb login --relogin` to force relogin)


True

In [2]:
import os
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.7.0 True
11.0
GCC 7.3


In [3]:
# 모듈 import
import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)

In [4]:
seed = 42

set_random_seed(seed, deterministic=False)
os.environ['PYTHONHASHSEED'] = str(seed)

In [5]:
from mmcv import Config

# config file 들고오기
baseline_cfg_path = './configs/gfl/gfl_x101_32x4d_fpn_dconv_c4-c5_mstrain_2x_coco.py'
cfg = Config.fromfile(baseline_cfg_path)

In [6]:
model_name = 'gfl_x101_32x4d_fpn_dconv_c4-c5_mstrain_2x_coco'
exp = 'basic'

# Folder to store model logs and weight files
job_folder = f'./working/exp[{exp}]_{model_name}'
cfg.work_dir = job_folder

# Change the wnd username and project name below
wnb_username = 'sunhyuk'
wnb_project_name = 'cv-3-bitcoin'

if not os.path.exists(job_folder):
    os.makedirs(job_folder)

print("Job folder:", job_folder)

Job folder: ./working/exp[basic]_gfl_x101_32x4d_fpn_dconv_c4-c5_mstrain_2x_coco


In [7]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'cv_train_1.json' # train json 정보
# cfg.data.train.pipeline[2]['img_scale'] = (1024,1024) # Resize
# cfg.train_pipeline[2]['img_scale'] = (1024,1024)
# cfg.test_pipeline[1]['img_scale'] = (1024,1024)
# cfg.train_pipeline[2]['ratio_range']=(1.0, 4.0)
# print(cfg.data.train.pipeline)
cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'cv_val_1.json' # test json 정보
# cfg.data.val.pipeline[1]['img_scale'] = (1024,1024) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
# cfg.data.test.pipeline[1]['img_scale'] = (1024,1024) # Resize
cfg.data.samples_per_gpu = 16
cfg.data.workers_per_gpu = 4

cfg.seed = seed #~~~
cfg.gpu_ids = [0]
#cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

# cfg.model.roi_head.bbox_head.num_classes = 10 ##클래수 수
cfg.model.bbox_head.num_classes = 10

cfg.evaluation = dict(
    interval=1,
    save_best='auto',
    metric='bbox',
    classwise=True,
)
#
cfg.optimizer = dict(type='SGD', lr=0.0005, momentum=0.9, weight_decay=0.0001, paramwise_cfg=dict(bias_lr_mult=2.0, bias_decay_mult=0.0))
# cfg.optimizer = dict(type='Adam', lr=3e-4)
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
# cfg.lr_config = dict(
#     policy='step',
#     warmup='linear',
#     warmup_iters=50,
#     warmup_ratio=0.001,
#     step=[8, 11]
# )

# cfg.runner.max_epochs = 12

wandb_cfg = dict(config={"epochs": cfg.runner.max_epochs, "batch_size": cfg.data.samples_per_gpu, "optimizer":cfg.optimizer, "model": cfg.model, "img_norm_cfg" : cfg.img_norm_cfg, "train_pipeline" : cfg.train_pipeline, "test_pipeline" : cfg.test_pipeline, "data" : cfg.data, "lr_config" : cfg.lr_config})

cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         interval=10,
         init_kwargs=dict(config=wandb_cfg, project=wnb_username,entity=wnb_project_name, name=f'[{exp}]_{model_name}')
        )
]
cfg.log_config.interval = 10
# cfg.checkpoint_config = 20
print(cfg.pretty_text)


dataset_type = 'CocoDataset'
data_root = 'data/coco/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=[(1333, 480), (1333, 800)],
        multiscale_mode='range',
        keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(
     

In [8]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

# dataset 확인
datasets[0]

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!



CocoDataset Train dataset with number of images 3677, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 2993  | 1 [Paper]     | 4685  | 2 [Paper pack]  | 681   | 3 [Metal]   | 711   | 4 [Glass]    | 733   |
| 5 [Plastic]       | 2211  | 6 [Styrofoam] | 970   | 7 [Plastic bag] | 3808  | 8 [Battery] | 136   | 9 [Clothing] | 378   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [9]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

/opt/ml/detection/baseline/mmdetection/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '
2022-04-05 04:47:01,201 - mmcv - INFO - initialize ResNeXt with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://resnext101_32x4d'}
2022-04-05 04:47:01,205 - mmcv - INFO - load model from: open-mmlab://resnext101_32x4d
2022-04-05 04:47:01,207 - mmcv - INFO - load checkpoint from openmmlab path: open-mmlab://resnext101_32x4d
2022-04-05 04:47:01,373 - mmcv - WARNING - The model and loaded state dict do not match exactly

missing keys in source state_dict: layer3.0.conv2.conv_offset.weight, layer3.0.conv2.conv_offset.bias, layer3.1.conv2.conv_offset.weight, layer3.1.conv2.conv_offset.bias, layer3.2.conv2.conv_offset.weight, layer3.2.conv2.conv_offset.bias, layer3.3.conv2.conv_offset.weight, layer3.3.conv2.conv_offs

In [10]:
# 모델 학습
train_detector(model, datasets[0], cfg=cfg, distributed=False, validate=True)

2022-04-05 04:47:04,904 - mmdet - INFO - Start running, host: root@642ef2a3bc92, work_dir: /opt/ml/detection/baseline/mmdetection/working/exp[basic]_gfl_x101_32x4d_fpn_dconv_c4-c5_mstrain_2x_coco
2022-04-05 04:47:04,905 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW   

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


/opt/conda/envs/mymmdet/lib/python3.7/site-packages/mmcv/runner/hooks/evaluation.py:222: UserWarning: runner.meta is None. Creating an empty one.
  warnings.warn('runner.meta is None. Creating an empty one.')


/opt/ml/detection/baseline/mmdetection/mmdet/models/dense_heads/gfl_head.py:256: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729047590/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  & (labels < bg_class_ind)).nonzero().squeeze(1)
2022-04-05 04:47:51,050 - mmdet - INFO - Epoch [1][10/230]	lr: 9.491e-06, eta: 6:30:58, time: 4.258, data_time: 0.316, memory: 27942, loss_cls: 0.0955, loss_bbox: 1.5667, loss_dfl: 0.7440, loss: 2.4063, grad_norm: 5.7443
2022-04-05 04:48:29,963 - mmdet - INFO - Epoch [1][20/230]	lr: 1.948e-05, eta: 6:13:29, time: 3.891, data_time: 0.029, memory: 27942, loss_cls: 0.1047, loss_bbox: 1.5478, loss_dfl: 0.7418, loss: 2.3943, grad_norm: 5.8920
2022-04-05 04:49:09,655 - mmdet - INFO - Epoch [1][30/230]	lr: 2.947e-05, eta: 6:09:36, time: 3.969, data_time: 0.028, memory: 27942, loss_cls: 0.1465, loss_bbox

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1206/1206, 6.7 task/s, elapsed: 181s, ETA:     0s

...

2022-04-05 12:09:10,817 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.49s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=7.78s).
Accumulating evaluation results...


2022-04-05 12:09:22,472 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.288
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.423
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.310
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.007
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.096
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.337
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.534
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.534
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.534
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.012
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.250
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=2.85s).



learning_rate,▂▃▅▇███████████████████████▂▂▂▂▂▂▂▂▂▂▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/grad_norm,▃▁▃▅▅▅▅▆█▆▅▅▇▅▅▆▅▅▄▅▇▅▇▆▅▅▅▃▄▄▄▃▄▃▅▄▄▄▅▆
train/loss,█▇▆▅▄▄▃▃▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁
train/loss_bbox,█▇▇▅▄▄▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▁▁▁▂▂▁▁▁▁▁▁▁
train/loss_cls,▆▄▁▄▅▄▆▅█▇▇▆▇▅▆▇▆▆▆▇▇▅▆▅▄▄▅▂▃▃▂▂▃▃▃▂▃▂▂▂
train/loss_dfl,█▇▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁
val/bbox_mAP,▁▂▃▃▄▅▅▆▆▆▆▇▇▇▇█████████
val/bbox_mAP_50,▁▂▃▄▅▆▆▆▆▆▇▇▇▇██████████
val/bbox_mAP_75,▁▁▂▃▄▅▅▆▆▆▆▇▇▇▇▇████████
val/bbox_mAP_l,▁▂▃▄▅▅▅▆▆▆▆▇▇▇▇█████████
